In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import xgboost as xgb
from sklearn.metrics import r2_score

In [2]:
start = datetime.datetime.now()
# df_k = pd.read_csv('filterpy kalman understanding/data_unscented1_kalman.csv')
df = pd.read_csv('data_linear_kalman.csv')
# df['unscented_kalman_var'] = df_k['unscented_kalman_var']
# df = pd.read_csv('data-training.csv')
print('Time taken to load the df is', datetime.datetime.now() - start)

Time taken to load the df is 0:00:14.516236


In [3]:
# df = df.loc[:1000,:]

# Features

#### Mid Price

In [4]:
df['wtd_mid_price0'] = ((df['askRate0'] * df['askSize0']) + (df['bidRate0'] * df['bidSize0']))/(df['bidSize0'] + df['askSize0'])
df['mid_price'] = (df['askRate0'] + df['bidRate0'])/2
df['diff'] = df['wtd_mid_price0'].diff()
df['lag1'] = df['diff'].shift(1)
df['lag2'] = df['diff'].shift(2)
df['lag3'] = df['diff'].shift(3)
df.shape

(2999999, 68)

# Singular Spectrum Analysis

In [5]:
X = df.loc[:, df.columns != 'y']
y = df.loc[:, df.columns == 'y']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle = False)
X_test = X_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [6]:
# start = datetime.datetime.now()

# from pyts.decomposition import SingularSpectrumAnalysis


# transformer = SingularSpectrumAnalysis(window_size = 9)
# transformer_train = pd.DataFrame(transformer.fit_transform(X_train['mid_price'].values.reshape(1,-1)).transpose(), columns = ['SSA1','SSA2','SSA3','SSA4','SSA5','SSA6','SSA7','SSA8','SSA9'])
# transformer_test = pd.DataFrame(transformer.transform(X_test['mid_price'].values.reshape(1,-1)).transpose(), columns = ['SSA1','SSA2','SSA3','SSA4','SSA5','SSA6','SSA7','SSA8','SSA9'])
# # transformer = pd.DataFrame(transformer.transform(X['mid_price'].values.reshape(1,-1)).transpose(), columns = ['SSA1','SSA2','SSA3','SSA4','SSA5','SSA6','SSA7','SSA8','SSA9'])

# X_train = pd.concat([X_train, transformer_train], axis = 1, ignore_index = True)
# X_test = pd.concat([X_test,  transformer_test], axis = 1, ignore_index = True)
# # X = pd.concat([X,  transformer], axis = 1, ignore_index = True)

# print('Time taken to run all the models is', datetime.datetime.now() - start)

In [7]:
start = datetime.datetime.now()

from pyts.decomposition import SingularSpectrumAnalysis


transformer = SingularSpectrumAnalysis(window_size = 25)
transformer_train = pd.DataFrame(transformer.fit_transform(X_train['mid_price'].values.reshape(1,-1)).transpose(), columns = ['SSA1','SSA2','SSA3','SSA4','SSA5','SSA6','SSA7','SSA8','SSA9','SSA10','SSA11','SSA12','SSA13','SSA14','SSA15','SSA16','SSA17','SSA18','SSA19','SSA120','SSA21','SSA22','SSA23','SSA24','SSA25'])
transformer_test = pd.DataFrame(transformer.transform(X_test['mid_price'].values.reshape(1,-1)).transpose(), columns = ['SSA1','SSA2','SSA3','SSA4','SSA5','SSA6','SSA7','SSA8','SSA9','SSA10','SSA11','SSA12','SSA13','SSA14','SSA15','SSA16','SSA17','SSA18','SSA19','SSA120','SSA21','SSA22','SSA23','SSA24','SSA25'])
# transformer_total = pd.DataFrame(transformer.transform(X['mid_price'].values.reshape(1,-1)).transpose(), columns = ['SSA1','SSA2','SSA3','SSA4','SSA5','SSA6','SSA7','SSA8','SSA9','SSA10','SSA11','SSA12','SSA13','SSA14','SSA15','SSA16','SSA17','SSA18','SSA19','SSA120','SSA21','SSA22','SSA23','SSA24','SSA25'])


X_train = pd.concat([X_train, transformer_train], axis = 1, ignore_index = True)
X_test = pd.concat([X_test,  transformer_test], axis = 1, ignore_index = True)
# X = pd.concat([X,  transformer_total], axis = 1, ignore_index = True)


print('Time taken to run all the models is', datetime.datetime.now() - start)

Time taken to run all the models is 0:02:34.656898


#### SMA

In [8]:
# df['SMA5_diff_lag_mid_price'] = df['diff_lag_mid_price'].rolling(window=5).mean()
# df['SMA5_askRate0'] = df['askRate0'].rolling(window=5).mean()
# df['SMA5_askSize0'] = df['askSize0'].rolling(window=5).mean()
# df['SMA5_bidRate0'] = df['bidRate0'].rolling(window=5).mean()
# df['SMA5_bidSize0'] = df['bidSize0'].rolling(window=5).mean()
# df.shape

#### EMA

In [9]:
# df['EMA_diff_lag_mid_price'] = df['diff_lag_mid_price'].ewm(com = 0.5, adjust = False).mean()
# df['EMA_askRate0'] = df['askRate0'].ewm(com = 0.5, adjust = False).mean()
# df['EMA_askSize0'] = df['askSize0'].ewm(com = 0.5, adjust = False).mean()
# df['EMA_bidRate0'] = df['bidRate0'].ewm(com = 0.5, adjust = False).mean()
# df['EMA_bidSize0'] = df['bidSize0'].ewm(com = 0.5, adjust = False).mean()
# df.shape

#### Lag

In [10]:
# df['lag1_mid_price0'] = df['diff_lag_mid_price'].shift(1)
# df['lag2_mid_price0'] = df['diff_lag_mid_price'].shift(2)
# df['lag3_mid_price0'] = df['diff_lag_mid_price'].shift(3)
# df['lag4_mid_price0'] = df['diff_lag_mid_price'].shift(4)

# df['lag1_mid_price0'] = df['mid_price'].shift(1)
# df['lag2_mid_price0'] = df['mid_price'].shift(2)
# df['lag3_mid_price0'] = df['mid_price'].shift(3)
# df['lag4_mid_price0'] = df['mid_price'].shift(4)

# df['lag1_wtd_mid_price0'] = df['wtd_mid_price0'].shift(1)
# df['lag2_wtd_mid_price0'] = df['wtd_mid_price0'].shift(2)
# df['lag3_wtd_mid_price0'] = df['wtd_mid_price0'].shift(3)
# df['lag4_wtd_mid_price0'] = df['wtd_mid_price0'].shift(4)

# df.shape

# Regressor

In [11]:
# X = df.loc[1:, df.columns != 'y']
# y = df.loc[1:, df.columns == 'y']

In [12]:
# X.shape

In [13]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle = False)

In [14]:
# X_train.shape

In [15]:
# X_test.shape

In [16]:
# start = datetime.datetime.now()

# learning_rate = [0.05,0.1,0.2]
# max_depth = [4,5,6]
# alpha = [10,11,12,13,14,15,16,17,18,19]
# colsample_bytree = [0.1,0.3, 0.8]
# colsample_bytree = [0.4,0.5,0.6]
# gamma = [0,1]
# min_child_weight = [0,1]
# n_estimators = [25,50,100]

# for i in gamma:
#     regressor = xgb.XGBRegressor(
#         objective ='reg:squarederror', 
#         colsample_bytree = 0.3,
#         learning_rate = 0.1,
#         max_depth = 5,
#         alpha = 15,
#         gamma = i,
#         n_estimators = 50)
    
#     regressor.fit(X_train,y_train)

#     y_pred = regressor.predict(X_test)
    
#     r2 = r2_score(y_test,y_pred)
    
#     print('R2 score for colsample_bytree value ', i, ' is ', r2)

# print('Time taken to run all the models is', datetime.datetime.now() - start)

In [17]:


regressor = xgb.XGBRegressor(
    objective ='reg:squarederror',
    colsample_bytree = 0.3,
    learning_rate = 0.1,
    max_depth = 5,
    alpha = 15,
    n_estimators = 50)

In [18]:
start = datetime.datetime.now()

regressor.fit(X_train,y_train)

y_pred = regressor.predict(X_test)
# y_pred = regressor.predict(X)


print('Time taken to train and predict the model is', datetime.datetime.now() - start)

Time taken to train and predict the model is 0:03:38.175302


# RSquared

In [19]:
r2 = r2_score(y_test,y_pred)
# r2 = r2_score(y,y_pred)

r2

0.13471536202864154

In [20]:
# 0.106